In [1]:
import pandas as pd
import networkx as nx
import itertools

In [2]:
df = pd.read_json("rawfilteredbydate.json")

In [3]:
df.head()

,caption,date
0,The Perlman Music Program Class of 2014,2014-08-22
1,Toby Perlman addresses the crowd,2014-08-22
2,Itzhak Perlman and Catherine Cho perform with...,2014-08-22
3,"Julian Beck, Lyn Barlow, Ted Ayala, Elinor He...",2014-08-22
4,Toby Perlman and guests,2014-08-22


In [4]:
df["caption"] = df['caption'].apply(lambda x: x.replace("\n",""))

In [5]:
df = df[df['caption'].apply(lambda x: len(x)<=250)]

In [117]:
restr = ["The", "Dr", "Verdana", "Helvetica" ,"Chairman","Director",
         "Guests","President","Hospital","Special Surgery","Executive Director"
         ,"New York","Museum","Photographs","Children","The Society","Dinner",
        "Board Member","Castle","President and CEO","Directors","Clockwise","Doubles","BFAnyc","Standing",
        "Susan Magazine","Trustees","Chicago","Yugoslavia","Steering Committee",
        "I Have A Dream"]

In [118]:
prereplace = ["friend","friends"]

In [119]:
def replaceWord(s):
    for i in range(len(prereplace)):
        s = s.replace(prereplace[i],"")
    return s

In [120]:
captions = map(lambda x:replaceWord(x),df.caption)

In [121]:
proper_noun_regex = r'([A-Z][A-Za-z-]*(,?\s+(\b(and|von|de|van der|van|der)\b))?(\s[A-Z][A-Za-z\'-]*)*)'

In [122]:
import re

In [134]:
G=nx.MultiGraph()
for caption in captions:
    name = []
    caption_temp  = re.findall(proper_noun_regex,caption)
    for j in range(len(caption_temp)):
        name_temp = caption_temp[j]
        if len(name_temp[0])>5 and len(name_temp[0].split(" ")) > 1 and len(name_temp[0].split(" ")) <=3:
            if name_temp[0] == "PatrickMcMullan":
                name.append("Patrick McMullan")
            elif name_temp[0] == "AnnieWatt":
                name.append("Annie Watt")
            elif "Mayor" in name_temp[0]:
                temp = name_temp[0].replace("Mayor ","")
                name.append(temp)
            else:
                name.append(name_temp[0])
        for word in restr:
            if word in name:
                name.remove(word)
    if len(name) > 2:
        _=[G.add_edge(c[0],c[1],weight=1) for c in itertools.combinations(name,2)]
    elif len(name) > 1:
        G.add_edge(name[0],name[1],weight=1)        

In [135]:
Gg = nx.Graph()
for u,v,data in G.edges_iter(data=True):
    if Gg.has_edge(u,v):
        Gg[u][v]['weight'] += 1
    else:
        Gg.add_edge(u, v, weight=1)

## degrees

In [136]:
degrees=pd.DataFrame(G.degree().values(),index=G.degree().keys()).sort(0,ascending=False)

In [137]:
degrees = degrees.reset_index()

In [138]:
degrees[180:220]

,index,0
180,Michel Witmer,97
181,Prince Dimitri,96
182,Jackie Harris Hochberg,96
183,Samuel Waxman,96
184,Olivia Chantecaille,96
185,Mary Alice Stephenson,96
186,Christian Leone,96
187,Celerie Kemble,95
188,Jonathan Tisch,95
189,David Lauren,95


In [139]:
degrees.columns = ["name","number"]

In [144]:
[i for i in degrees.name if " and " in i]

[u'Chairman and CEO',
 u'Kelly, and Dr',
 u'Cynthia and Dr',
 u'Sex, and Drugs',
 u'Ambassador and Mrs',
 u'Barbados, and Hon',
 u'Hailey and Mama',
 u'Fish and Wildlife',
 u'Castle, and Dr',
 u'Vecchio, and Claudio',
 u'Writer and Director',
 u'Arts and Design',
 u'Colonel and Mrs',
 u'Strickland, and Dr',
 u'Sophie and MikeTansey',
 u'Nancy and Dr',
 u'CEO and President',
 u'Art and Design',
 u'Hamilton, and Mr',
 u'Zoe, and Grover',
 u'Founder and Chairman',
 u'Gordon and Co',
 u'Justin and Evan',
 u'Ban, and Gala',
 u'Carolina and Dr',
 u'Music and Art',
 u'Salvo, and Annemarie',
 u'Founder and CEO',
 u'Zuylen and Mr',
 u'Cohn, and Dr',
 u'CEO and Chairman',
 u'Parke and H',
 u'Chris and Grace',
 u'Graham and Grady',
 u'Co-Chairs and Town',
 u'President and CFO',
 u'Logistics and Distribution',
 u'Monkey and Me',
 u'Sweden, and Mrs',
 u'Matt and Michel',
 u'Joseph and Luis',
 u'Rigas, and Master',
 u'Finance and Enterprise',
 u'Pastels, and Prints',
 u'Gudrun and Dr',
 u'Philadelph

In [141]:
degreeres = map(lambda x,y: (x,y),degrees.name[0:100], degrees.number[0:100])

In [142]:
import pickle

In [143]:
with open('degreeres.pickle', 'wb') as f:
    pickle.dump(degreeres, f)

In [14]:
pr=nx.pagerank(Gg)
pranalysis=pd.DataFrame(pr.values(),index=pr.keys()).sort(0,ascending=False)